# Model evaluation

Now that we have a model trained that seems to have learned well, we can examine it to see how well it really learned.

In [1]:
import yaml
import torch
from ataarangi.models import TransformerModel, RNNModel
from ataarangi.data import SequenceTokenizer, load_data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Initialize tokenizers
tokenizer = SequenceTokenizer('../data/worldstate_tokens.txt', '../data/tokens.txt')

In [4]:
tokenizer.token_map

{'[PAD]': 0,
 '[SOS]': 1,
 '[SELECTED]': 2,
 '[NOT_SELECTED]': 3,
 '[COLOUR_RED]': 4,
 '[COLOUR_BLUE]': 5,
 '[COLOUR_GREEN]': 6,
 '[COLOUR_YELLOW]': 7,
 '[COLOUR_BLACK]': 8,
 '[COLOUR_WHITE]': 9,
 '[COLOUR_BROWN]': 10,
 '[COLOUR_PINK]': 11,
 '[HEIGHT_1]': 12,
 '[HEIGHT_2]': 13,
 '[HEIGHT_3]': 14,
 '[HEIGHT_4]': 15,
 '[HEIGHT_5]': 16,
 '[HEIGHT_6]': 17,
 '[HEIGHT_7]': 18,
 '[HEIGHT_8]': 19,
 '[HEIGHT_9]': 20,
 '[HEIGHT_10]': 21,
 '[CLS]': 22,
 'rākau': 23,
 'te': 24,
 'ngā': 25,
 'me': 26,
 'mā': 27,
 'kākāriki': 28,
 'kōwhai': 29,
 'kikorangi': 30,
 'parauri': 31,
 'pango': 32,
 'whero': 33,
 'māwhero': 34,
 'iti': 35,
 'nui': 36,
 'hāunga': 37,
 'katoa': 38,
 'taha': 39,
 'kei': 40,
 'mauī': 41,
 'matau': 42,
 'ki': 43,
 'tawhiti_rawa': 44,
 'rawa': 45,
 'e': 46,
 'rua': 47,
 'waenganui': 48,
 'i': 49,
 'toru': 50,
 'tuarua': 51,
 'mai': 52,
 '[EOS]': 53}

In [5]:
def load_model(path, params):
    model = RNNModel(**params)
    model.load_state_dict(torch.load(path))
    return model

best_model_params = {
    'tokenizer': tokenizer,
    'embed_size': 128,
    'hidden_size': 512,
    'num_layers': 4
}

best_model = load_model(
    '../models/lr=0.001-num_layers=4-embed_size=128-hidden_size=512-dropout=0.0.pth',
    best_model_params
)

In [6]:
train_data, dev_data = load_data('../data/train_set.csv', '../data/dev_set.csv')

In [7]:
tokens = tokenizer.tokenize(train_data['rākau'][0])
tokens_tensor = torch.tensor(tokens, dtype=torch.long).to(device)  # Ensure the tensor is of type long
generated_sequence = torch.argmax(best_model.generate(tokens), axis=-1)[0].tolist()
print(generated_sequence)

[3, 5, 15, 2, 9, 19, 22, 24, 23, 27, 53]


In [8]:
train_data['rākau'][0]

[{'color': 'blue', 'height': 4, 'location': 1, 'selected': False},
 {'color': 'white', 'height': 8, 'location': 2, 'selected': True}]

In [9]:
print(tokenizer.decode(generated_sequence))

[NOT_SELECTED] [COLOUR_BLUE] [HEIGHT_4] [SELECTED] [COLOUR_WHITE] [HEIGHT_8] [CLS] te rākau mā [EOS]
